In [1]:
# David Laub dlaub@ucsd.edu
# Lin Chou lic130@pitt.edu

In [2]:
from pathlib import Path
from genoray.exprs import is_snp,is_indel
from genoray import VCF
import polars as pl
import awkward as ak
import pyranges as pr
from seqpro._ragged import Ragged
from einops import repeat
import numpy as np

/home/lic130/.local/lib/python3.12/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
ddir = Path("/home/lic130/scratch/hmp01/hmp01_patient_count")

In [4]:
# a function for deduplication by getting the union
def get_dedup_genos(vcf, sample_sheet):
    genos = vcf.read("16") == 1
    n_cases = ss['case_id'].n_unique()
    cases = []
    dedup_genos = np.empty((n_cases, 2, genos.shape[-1]), np.uint8)
    for i, ((case_id,), df) in enumerate(ss.group_by('case_id', maintain_order=True)):
        idx = df['index']
        dedup_genos[i] = genos[idx].any(0)
        cases.append(case_id)
    return cases, dedup_genos

# New WGS Data $n_{\text{runs}}$=10,203, $n_{\text{cases}}$=8,913

In [5]:
exons = pr.read_bed(str(ddir / "VPS9D1-AS1_exons.bed")).drop("Strand").merge()
n_orfs = pr.read_bed(str(ddir / "VPS9D1-AS1_nORFs.bed")).merge()
tr = pr.read_bed(str(ddir / "VPS9D1-AS1_TR.bed")).merge()

In [6]:
#vcf = VCF(ddir / "gdc.gatk4_wgs.chou_orf143.norm.bcf")
vcf = VCF(ddir /  "indel_dir" / "gdc.gatk4_wgs.chou_orf143.norm.bcf")
if not vcf._valid_index():
    vcf._write_gvi_index()
vcf._load_index()
ss = (
    pl.read_csv("/home/lic130/scratch/hmp01/hmp01_patient_count/gdc_gatk4_wgs_sample_sheet.tsv", separator="\t")
    .rename(lambda c: c.lower().replace(" ", "_"))
    .with_columns(pl.col("case_id").str.split(", ").list.first())
    .join(
        pl.DataFrame({"file_id": vcf.available_samples}).with_row_index(),
        on="file_id",
        maintain_order="right",
    )
)
assert ss.height == len(vcf.available_samples)
ss.head()

2025-07-24 15:05:12.681 | INFO     | genoray._vcf:_load_index:1059 - Loading genoray index.


file_id,file_name,data_category,data_type,project_id,case_id,sample_id,sample_type,index
str,str,str,str,str,str,str,str,u32
"""0000ff8f-77b1-48ce-bbee-b12b5d…","""CPTAC-3.46727fd6-e725-4a03-a44…","""Simple Nucleotide Variation""","""Raw Simple Somatic Mutation""","""CPTAC-3""","""C207624""","""SM-JU34N, 1105207""","""Primary Tumor, Blood Derived N…",0
"""0005b113-ab59-4467-8e54-ff2ef3…","""TCGA-MESO.89d209b8-f740-47d2-a…","""Simple Nucleotide Variation""","""Raw Simple Somatic Mutation""","""TCGA-MESO""","""TCGA-MQ-A4LV""","""TCGA-MQ-A4LV-10B, TCGA-MQ-A4LV…","""Blood Derived Normal, Primary …",1
"""0005fbc7-794a-4ac8-911b-d4c209…","""TCGA-TGCT.e048949b-e586-4cf5-9…","""Simple Nucleotide Variation""","""Raw Simple Somatic Mutation""","""TCGA-TGCT""","""TCGA-2G-AAHC""","""TCGA-2G-AAHC-10A, TCGA-2G-AAHC…","""Blood Derived Normal, Primary …",2
"""001611e0-2525-4509-9684-bc7c4c…","""TCGA-HNSC.34719f72-4510-4128-a…","""Simple Nucleotide Variation""","""Raw Simple Somatic Mutation""","""TCGA-HNSC""","""TCGA-HD-A4C1""","""TCGA-HD-A4C1-01A, TCGA-HD-A4C1…","""Primary Tumor, Blood Derived N…",3
"""0017b1f6-4900-4fe0-8419-79bd1f…","""REBC-THYR.f3f12e32-95c5-4989-9…","""Simple Nucleotide Variation""","""Raw Simple Somatic Mutation""","""REBC-THYR""","""REBC-ACEV""","""REBC-ACEV-NT1-A, REBC-ACEV-TTM…","""Solid Tissue Normal, Metastati…",4


In [7]:
### deduplication
dedup_res = get_dedup_genos(vcf,ss)
# samples are goupred by case_id
# store case id
case_id = dedup_res[0]
# store the dedup genotype array
dedup_genos = dedup_res[1]

In [8]:
### check the results
# before dedup.
vcf.read("16").sum((0, 1))

array([3, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1])

In [9]:
# after
dedup_genos.sum((0, 1))
# no 0 good.

array([2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1], dtype=uint64)

In [10]:
# are the mutation counts generally smaller in the dedup data because samples are merged?
geno_cnts = vcf.read("16").sum((0, 1))
dedup_cnts = dedup_genos.sum((0, 1))
np.testing.assert_equal(geno_cnts > 0, dedup_cnts > 0)
assert (dedup_cnts <= geno_cnts).all()

In [11]:
tmb = (
    pl.read_csv(
        ddir / "gdc.gatk4_wgs.chr16_tmb.tsv",
        separator="\t",
        has_header=False,
        new_columns=["file_id", "tmb"],
    )
    .join(ss, on="file_id", maintain_order="right")
    .group_by("case_id", maintain_order=True)
    .agg(pl.col("tmb").mean())
)
assert dedup_genos.shape[0] == tmb.height
tmb.head()

case_id,tmb
str,f64
"""C207624""",111.666667
"""TCGA-MQ-A4LV""",85.0
"""TCGA-2G-AAHC""",36.0
"""TCGA-HD-A4C1""",698.0
"""REBC-ACEV""",33.0


In [12]:
in_exons = vcf._index.gr.overlap(exons)
in_orf = in_exons.overlap(n_orfs)
out_orf = in_exons.overlap(n_orfs, invert=True)
in_orf_no_tr = in_orf.overlap(tr, invert=True)
out_orf_no_tr = out_orf.overlap(tr, invert=True)
assert len(in_exons) != 0, "No variants in exons"

In [13]:
# in_exons = vcf._index.gr.overlap(exons)
# in_orf = in_exons.overlap(n_orfs)
# out_orf = in_exons.overlap(n_orfs, invert=True)
# in_orf_no_tr = in_orf.overlap(tr, invert=True)
# out_orf_no_tr = out_orf.overlap(tr, invert=True)

in_orf_cnts = dedup_genos[..., in_orf.index].sum((1, 2))
#out_orf_cnts = dedup_genos[..., out_orf.index].sum((1, 2))
in_orf_no_tr_cnts = dedup_genos[..., in_orf_no_tr.index].sum((1, 2))


# out_orf
if len(out_orf) == 0:
    out_orf_cnts = np.array(0.)
else:
    out_orf_cnts = dedup_genos[..., out_orf.index].sum((1, 2))
# out_orf_no_tr
if len(out_orf_no_tr) == 0:
    out_orf_no_tr_cnts = np.array(0.)
else:
    out_orf_no_tr_cnts = dedup_genos[..., out_orf_no_tr.index].sum((1, 2))

##
in_orf_cnts.sum(), out_orf_cnts.sum(), in_orf_no_tr_cnts.sum(), out_orf_no_tr_cnts.sum()

(np.uint64(15), np.float64(0.0), np.uint64(10), np.float64(0.0))

In [14]:
## use dedup_genos to find indels (patient, ploidy, mutation), mataches sample sheet or tmb table
# patients with mutation in ORF
patient_in_orf = dedup_genos[..., in_orf.index]
patient_in_orf_non0 = np.any(patient_in_orf != 0, axis=(1, 2))
patient_in_orf_non0
# patients with mutation outside the ORF
# patient_out_orf = dedup_genos[..., out_orf.index]
# patient_out_orf_non0 = np.any(patient_out_orf != 0, axis=(1, 2))
# make a dataframe
mut_loc_df = pl.DataFrame({
    "case_id": case_id,
    "in_orf_mutation": patient_in_orf_non0
    # "out_orf_mutation": patient_out_orf_non0
})
# a thrid col for patients that have both types
# mut_loc_df = df.with_columns(
#     (pl.col("in_orf_mutation") & pl.col("out_orf_mutation")).alias("both")
# )
mut_loc_df

case_id,in_orf_mutation
str,bool
"""C207624""",false
"""TCGA-MQ-A4LV""",false
"""TCGA-2G-AAHC""",false
"""TCGA-HD-A4C1""",false
"""REBC-ACEV""",false
…,…
"""TCGA-24-2029""",false
"""TCGA-BA-A4II""",false
"""TCGA-A5-A0G1""",false


In [15]:
## combine tmb table and the mutation
tmb_mut_loc = tmb.join(mut_loc_df, on="case_id", how="left")
tmb_mut_loc

case_id,tmb,in_orf_mutation
str,f64,bool
"""C207624""",111.666667,false
"""TCGA-MQ-A4LV""",85.0,false
"""TCGA-2G-AAHC""",36.0,false
"""TCGA-HD-A4C1""",698.0,false
"""REBC-ACEV""",33.0,false
…,…,…
"""TCGA-24-2029""",84.0,false
"""TCGA-BA-A4II""",421.0,false
"""TCGA-A5-A0G1""",35202.0,false


In [16]:
## add cancer type

# download clinical data from GDC > cases > clinical
# /home/lic130/scratch/hmp01/hmp01_patient_count/clinical.cohort_20250710
clinical_df = pl.read_csv("/home/lic130/scratch/hmp01/hmp01_patient_count/clinical.cohort_20250710/clinical.tsv", separator="\t")
clinical_df = clinical_df.select(["cases.submitter_id","cases.disease_type","cases.primary_site"])

## left join
# rename column names
clinical_df.columns = ["case_id", "disease_type", "primary_site"]
# 
tmb_mut_loc_cancer = tmb_mut_loc.join(clinical_df, on="case_id", how="left")

## check
# is there NA?
tmb_mut_loc_cancer.select(
    (pl.col("disease_type").is_null() | pl.col("primary_site").is_null()).any()
)
# no NA. good
# shape
tmb_mut_loc_cancer.shape
## remove duplicated rows
tmb_mut_loc_cancer = tmb_mut_loc_cancer.unique()
tmb_mut_loc_cancer

## order based on original TMB
df2 = tmb
# Add a helper column that encodes the desired order
df2 = df2.with_row_count(name="order")
# # Join and sort
ordered_df1 = tmb_mut_loc_cancer.join(df2, on="case_id").sort("order").select(tmb_mut_loc_cancer.columns)
tmb_mut_loc_cancer = ordered_df1
tmb_mut_loc_cancer


# make sure the new table still have the same order
assert (tmb_mut_loc_cancer["case_id"] == tmb["case_id"]).all()

## export
tmb_mut_loc_cancer.write_csv(
    ddir / "gdc.indel.tmb_mut_loc_cancer.tsv",
        separator="\t")

/tmp/ipykernel_2242123/575206636.py:29: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  df2 = df2.with_row_count(name="order")


In [17]:
## check composition of patients with in orf mutation
# Filter
filtered = tmb_mut_loc_cancer.filter(
    (pl.col("in_orf_mutation") == True) #& (pl.col("out_orf_mutation") == False)
)
# Count per disease_type
counts = filtered.group_by("disease_type").agg([
    pl.count().alias("count")
])

# Calculate percentages
result = counts.with_columns(
    (pl.col("count") / pl.col("count").sum() * 100).alias("percentage")
)
# Sort by count descending
result = result.sort("count", descending=True)
result

/tmp/ipykernel_2242123/1688864765.py:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count")


disease_type,count,percentage
str,u32,f64
"""Adenomas and Adenocarcinomas""",12,80.0
"""Squamous Cell Neoplasms""",1,6.666667
"""Gliomas""",1,6.666667
"""Transitional Cell Papillomas a…",1,6.666667


In [18]:
### location of mutation for Adenomas and Adenocarcinomas
## only do cancer type-specific analysis for Adenomas and Adenocarcinomas because the sample size of others are extremely small, meaning low power
# get the index for cancer type of interest
idx_cancer_type = (tmb_mut_loc_cancer["disease_type"] == "Adenomas and Adenocarcinomas").to_numpy().nonzero()[0]
idx_cancer_type
# subset based on cancer type
PH01 = dedup_genos[idx_cancer_type]

# subset based on location of mutation
in_orf_cnts = PH01[:,:, in_orf.index].sum((1, 2))
in_orf_no_tr_cnts = PH01[:,:, in_orf_no_tr.index].sum((1, 2))
if len(out_orf) == 0:
    out_orf_cnts = np.array(0.)
else:
    out_orf_cnts = dedup_genos[..., out_orf.index].sum((1, 2))

if len(out_orf_no_tr) == 0:
    out_orf_no_tr_cnts = np.array(0.)
else:
    out_orf_no_tr_cnts = dedup_genos[..., out_orf_no_tr.index].sum((1, 2))

in_orf_cnts.sum(), out_orf_cnts.sum(), in_orf_no_tr_cnts.sum(), out_orf_no_tr_cnts.sum()

(np.uint64(12), np.float64(0.0), np.uint64(9), np.float64(0.0))

In [19]:
### mutation - count and coord
in_exons = vcf._index.gr.overlap(exons)
in_orf = in_exons.overlap(n_orfs)
out_orf = in_exons.overlap(n_orfs, invert=True)
in_orf_no_tr = in_orf.overlap(tr, invert=True)
# out_orf_no_tr = out_orf.overlap(tr, invert=True)

### make a pl data frame

# conver panda to polars
in_exons_df=pl.from_pandas(
    # make panda df
    in_exons.df)

### count of each mutation
in_exons_cnts = dedup_genos[..., in_exons.index].sum((0, 1))
# add to the df
# in_exons_df = 
in_exons_df =in_exons_df.with_columns(pl.Series("count", in_exons_cnts))


### annotate location

## in_orf
# Create a list of None (nulls)
new_col = [None] * in_exons_df.height
# Assign "yes" to desired indices
for idx in in_orf.index.to_numpy():
    new_col[idx] = "yes"
# Add the column to the DataFrame
in_exons_df = in_exons_df.with_columns(pl.Series("in_orf", new_col))
in_exons_df

# ## out_orf
# # Create a list of None (nulls)
# new_col = [None] * in_exons_df.height
# # Assign "yes" to desired indices
# for idx in out_orf.index.to_numpy():
#     new_col[idx] = "yes"
# # Add the column to the DataFrame
# in_exons_df = in_exons_df.with_columns(pl.Series("out_orf", new_col))
# in_exons_df

## in_orf_no_tr
# Create a list of None (nulls)
new_col = [None] * in_exons_df.height
# Assign "yes" to desired indices
for idx in in_orf_no_tr.index.to_numpy():
    new_col[idx] = "yes"
# Add the column to the DataFrame
in_exons_df = in_exons_df.with_columns(pl.Series("in_orf_no_tr", new_col))
in_exons_cnts


## out_orf_no_tr
# none

# ## filter out count = 0
# in_exons_df = in_exons_df.filter(pl.col("count") != 0)

## export
in_exons_df.write_csv(
    ddir / "in_exons_indel.tsv",
        separator="\t")

# HMF

In [20]:
exons = pr.read_bed(str(ddir / "VPS9D1-AS1_exons.hg19.bed")).drop("Strand").merge()
n_orfs = pr.read_bed(str(ddir / "VPS9D1-AS1_nORFs.hg19.bed")).merge()
tr = pr.read_bed(str(ddir / "VPS9D1-AS1_TR.hg19.bed")).merge()
exons.Chromosome = "16"
n_orfs.Chromosome = "16"
tr.Chromosome = "16"

In [21]:
tmb = pl.read_csv(ddir / "hmf.chr16.tmb.tsv", separator="\t")
tmb.head()

case_id,tmb
str,i64
"""WIDE01010971T""",78
"""ACTN01020002T""",141
"""DRUP01070016T""",63
"""CPCT02040051T""",1144
"""CPCT02030455T""",465


In [22]:
vcf = VCF(ddir /"indel_dir"/ "hmf.vps9d1-as1.vaf.norm.bcf")#.set_samples(tmb["#IID"])

if not vcf._valid_index():
    vcf._write_gvi_index()
vcf._load_index()

2025-07-24 15:05:15.049 | INFO     | genoray._vcf:_load_index:1059 - Loading genoray index.


In [23]:
genos = vcf.read("16") == 1
assert genos.shape[0] == tmb.height

In [24]:
norm_genos = genos / tmb["tmb"].to_numpy()[:, None, None]
norm_genos = np.nan_to_num(norm_genos, nan=0, posinf=0, neginf=0)

In [25]:
in_exons = vcf._index.gr.overlap(exons)
in_orf = in_exons.overlap(n_orfs)
out_orf = in_exons.overlap(n_orfs, invert=True)
in_orf_no_tr = in_orf.overlap(tr, invert=True)
out_orf_no_tr = out_orf.overlap(tr, invert=True)

assert len(in_exons) != 0, "No variants in exons"

In [26]:
in_orf_cnts = genos[..., in_orf.index].sum((1, 2))
# out_orf_cnts = genos[..., out_orf.index].sum((1, 2))
in_orf_no_tr_cnts = genos[..., in_orf_no_tr.index].sum((1, 2))


if len(out_orf) == 0:
    out_orf_cnts = np.array(0)
else:
    out_orf_cnts = genos[..., out_orf.index].sum((1, 2))


if len(out_orf_no_tr) == 0:
    out_orf_no_tr_cnts = np.array(0)
else:
    out_orf_no_tr_cnts = genos[..., out_orf_no_tr.index].sum((1, 2))

in_orf_cnts.sum(), out_orf_cnts.sum(), in_orf_no_tr_cnts.sum(), out_orf_no_tr_cnts.sum()

(np.int64(8), np.int64(0), np.int64(5), np.int64(0))

In [27]:
## patients with mutation in ORF
patient_in_orf = genos[..., in_orf.index]
patient_in_orf_non0 = np.any(patient_in_orf != 0, axis=(1, 2))
patient_in_orf_non0
# # patients with mutation outside the ORF
# patient_out_orf = dedup_genos[..., out_orf.index]
# patient_out_orf_non0 = np.any(patient_out_orf != 0, axis=(1, 2))
# make a dataframe
df = pl.DataFrame({
    "case_id": tmb["case_id"],
    "in_orf_mutation": patient_in_orf_non0
    # ,
    # "out_orf_mutation": patient_out_orf_non0
})
df
# # a thrid col for patients that have both types
# mut_loc_df = df.with_columns(
#     (pl.col("in_orf_mutation") & pl.col("out_orf_mutation")).alias("both")
# )
mut_loc_df = df

In [28]:
## combine tmb table and the mutation
tmb_mut_loc = tmb.join(mut_loc_df, on="case_id", how="left")
tmb_mut_loc

case_id,tmb,in_orf_mutation
str,i64,bool
"""WIDE01010971T""",78,false
"""ACTN01020002T""",141,false
"""DRUP01070016T""",63,false
"""CPCT02040051T""",1144,false
"""CPCT02030455T""",465,false
…,…,…
"""CPCT02040045T""",3230,false
"""CPCT02020535T""",2523,false
"""WIDE01011223T""",7250,false
